In [1]:
import os
import sys

ROOT_PATH=os.path.realpath('../..')
if not ROOT_PATH in sys.path:
    sys.path.append(ROOT_PATH)

from lib.etl.ons import metadata as ons_metadata, transform as ons_transform

Set Papermill parameters

In [2]:
dataset: str = 'unem'   # ID for ONS time series dataset to process
url: str = 'https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peoplenotinwork/unemployment/datasets/claimantcountandvacanciesdataset/current/unem.csv'

Derive filenames

In [3]:
RAW_CSV=f'../../data/raw/{dataset}.csv'
METADATA_FILE = f'../../data/metadata/codes/{dataset}-codes.csv'
DATA_FILE = f'../../data/processed/{dataset}.csv'

Make sure the directories exist

In [4]:
os.makedirs(os.path.dirname(METADATA_FILE), exist_ok=True)
os.makedirs(os.path.dirname(DATA_FILE), exist_ok=True)

Extract metadata

In [5]:
ons_metadata(RAW_CSV).tocsv(METADATA_FILE, lineterminator='\n')

In [6]:
ons_transform(RAW_CSV).tocsv(DATA_FILE, lineterminator='\n')